In [15]:
#Get matchs por puuid and matchs id

In [16]:
import requests
import json
import os
import re
import time
from minio import Minio
from minio.error import S3Error
import io


#MINIO CONFIGS
minio_endpoint = "minio:9000" # Mudar para o IP quando rodar no Docker
minio_access_key = "tcc_fia"
minio_secret_key = "tcc_fia"
minio_bucket_name  = "raw"


def get_minio_client():
    minio_client = Minio(minio_endpoint,
                            access_key=minio_access_key,
                            secret_key=minio_secret_key,
                            secure=False)

    return minio_client

def get_puuid_by_name(player_name:str):
    base_url = 'https://americas.api.riotgames.com'
    endpoint = f"/riot/account/v1/accounts/by-riot-id/{player_name}/br1"
    token = "RGAPI-437f5d9b-23f6-48cf-860b-868a828904ec"
    request_url = f"{base_url}{endpoint}"
    headers = {
        "X-Riot-Token": f"{token}"
        }
    
    response = requests.get(request_url, headers=headers)

    if response.status_code == 200:
        return json.loads(response.text)
    else:
        print("Error API Status-Code: ", response.status_code)


def get_match_by_puuid(puuid:str):
    base_url = 'https://americas.api.riotgames.com'
    endpoint = f"/lol/match/v5/matches/by-puuid/{puuid}/ids"
    token = "RGAPI-437f5d9b-23f6-48cf-860b-868a828904ec"
    request_url = f"{base_url}{endpoint}"
    headers = {
        "X-Riot-Token": f"{token}"
        }
    
    response = requests.get(request_url, headers=headers)

    if response.status_code == 200:
        return json.loads(response.text)
    else:
        print("Error API Status-Code: ", response.status_code)

def get_matchs_details(match_id:str):
    
    base_url = 'https://americas.api.riotgames.com'
    endpoint = f"/lol/match/v5/matches/{match_id}/timeline"
    token = "RGAPI-437f5d9b-23f6-48cf-860b-868a828904ec"
    request_url = f"{base_url}{endpoint}"
    headers = {
        "X-Riot-Token": f"{token}"
        }
    
    response = requests.get(request_url, headers=headers)

    if response.status_code == 200:
        return json.loads(response.text)
    else:
        print("Error API Status-Code: ", response.status_code)

def check_folder_exists(minio_bucket_name, folder_path):
    
    minio_client = get_minio_client()
    try:
        # Lista todos objetos na pasata
        objects = minio_client.list_objects(minio_bucket_name, prefix=folder_path, recursive=False)
        
        folders = {obj.object_name.split("/")[0] for obj in objects if obj.is_dir}
        
        # Retorno true se encontrar as pastas
        return folder_path.rstrip("/") in folders

    except S3Error as err:
        return False  # Retorna False se não existir

def create_folders(minio_bucket_name, folder_path):
    minio_client = get_minio_client()
    try:
        minio_client.put_object(minio_bucket_name, f"{folder_path}/", io.BytesIO(b''), 0)
        print(f"O caminho no BUCKET {minio_bucket_name}/'{folder_path}' foi criado.")

    except S3Error as err:
        print(f"Error: {err}")

def upload_json_to_minio(bucket_name, folder_path, filename, data):
    minio_client = get_minio_client()
    if not check_folder_exists(minio_bucket_name,folder_path):
        create_folders(minio_bucket_name, folder_path)
        print(f'Folder {bucket_name}/{folder_path} criado com sucesso')
    try:
        # CONVERTE JSON PARA BYTES 
        json_bytes = json.dumps(data, ensure_ascii=False).encode('utf-8')

        # FAZ UPLOAD TO MINIO 
        object_name = os.path.join(folder_path, filename)
        minio_client.put_object(bucket_name, object_name, io.BytesIO(json_bytes), len(json_bytes))

        print(f"UPLOAD JSON  '{filename}' COM SUCESSO.")

    except S3Error as err:
        print(f"Error: {err}")

In [17]:
player_puuid = get_puuid_by_name('FGS theone')['puuid']
#print(player_puuid)

match_list = get_match_by_puuid(player_puuid)
#print(match_list)

In [18]:
for match in match_list:
    match_detail = get_matchs_details(match)
    upload_json_to_minio(minio_bucket_name
                         ,'matchs_raw'
                         , match 
                         ,match_detail)

O caminho no BUCKET raw/'matchs_raw' foi criado.
Folder raw/matchs_raw criado com sucesso
UPLOAD JSON  'BR1_2852249864' COM SUCESSO.
UPLOAD JSON  'BR1_2852143434' COM SUCESSO.
UPLOAD JSON  'BR1_2852114571' COM SUCESSO.
UPLOAD JSON  'BR1_2852086930' COM SUCESSO.
UPLOAD JSON  'BR1_2851892756' COM SUCESSO.
UPLOAD JSON  'BR1_2851866104' COM SUCESSO.
UPLOAD JSON  'BR1_2851548624' COM SUCESSO.
UPLOAD JSON  'BR1_2851522196' COM SUCESSO.
UPLOAD JSON  'BR1_2851416086' COM SUCESSO.
UPLOAD JSON  'BR1_2851116533' COM SUCESSO.
UPLOAD JSON  'BR1_2851019414' COM SUCESSO.
UPLOAD JSON  'BR1_2850938568' COM SUCESSO.
UPLOAD JSON  'BR1_2849610882' COM SUCESSO.
UPLOAD JSON  'BR1_2847875584' COM SUCESSO.
UPLOAD JSON  'BR1_2847854076' COM SUCESSO.
UPLOAD JSON  'BR1_2847650589' COM SUCESSO.
UPLOAD JSON  'BR1_2847609342' COM SUCESSO.
UPLOAD JSON  'BR1_2847501020' COM SUCESSO.
UPLOAD JSON  'BR1_2847478337' COM SUCESSO.
UPLOAD JSON  'BR1_2847370319' COM SUCESSO.
